In [ ]:
!pip install confluent_kafka

In [ ]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import sys

import json
import logging
from IPython.display import display, clear_output

import time

Consumer settings:

In [ ]:
conf = {
        'bootstrap.servers': 'amdsehubns.servicebus.windows.net:9093', 
        'security.protocol': 'SASL_SSL',
        #replace below item with your environments certificate path, you can keep it as .pem or rename to .crt' both works:
        'ssl.ca.location': 'C:\\Certs\\cacert.pem',
        'sasl.mechanism': 'PLAIN',
        #Dont Change here:
        'sasl.username': '$ConnectionString',
        #replace below item with your Shared Access Key Connection string of your EventHubs Namespace:
        'sasl.password':'Endpoint=sb://amdsehubns.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=zVXlYP9+5raLvD4Qrcl2UHqYwCPXEOUcpZtYWA99wZI=',      
        #replace below item with your consumer group
        'group.id': 'kafka',
        'session.timeout.ms': 60000,
        'default.topic.config': {'auto.offset.reset': 'smallest'},
         #replace below item with your prefered cliend id
        'client.id': 'kafkaconsumerpython'
    }
#replace below item with your event hubs name
topic = ["hello"]

Create logger for consumer (logs will be emitted when poll() is called)

In [ ]:
logger = logging.getLogger('consumer')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)-15s %(levelname)-8s %(message)s'))
logger.addHandler(handler)

Create Consumer object and subscribe it to the topic(event hubs)

In [ ]:
consumer = Consumer(conf, logger=logger)

In [ ]:
consumer.subscribe(topic)

Start polling messages:

In [ ]:
while True:    
    message = consumer.poll(timeout=100.0)
    if message is None:
        clear_output(wait=True)
        display('No Message | cts: '+ time.ctime(int(time.time())))
        continue
    if message.error():
        # Error or event
        if message.error().code() == KafkaError._PARTITION_EOF:
            # End of partition event
            display('%% %s [%d] reached end at offset %d\n' % (message.topic(), message.partition(),message.offset()))
        else:
            # Error
            raise KafkaException(message.error())
            break
    clear_output(wait=True)
    display(str(message.value()) + ' cts: '+ time.ctime(int(time.time())))